In [266]:
#imports
import pennylane as qml
from pennylane import numpy as np
import networkx as nx

In [267]:
n_nodes = 9

# graphs for training
graph_0 = [(0, 7), (1, 5), (1, 6), (1, 7), (2, 4), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (4, 7), (5, 6), (5, 7), (6, 7), (6, 8), (7, 8)]
graph_1 = [(0, 4), (1, 4), (1, 5), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 4), (3, 5), (4, 5), (4, 6), (4, 8), (5, 7)]

## grid graph
#graph_0 = nx.Graph([(0, 1), (0, 3), (1, 2), (1, 4), (2, 5), (3, 4), (3, 6), (4, 5), (4, 7), (5, 8), (6, 7), (7, 8)])
#graph_1 = nx.Graph([(0, 2), (0, 7), (1, 5), (1, 6), (1, 8), (2, 5), (2, 8), (3, 4), (3, 7), (4, 6), (4, 8), (7, 8)])
## cycle graph
#graph_0 = nx.Graph([(0, 1), (0, 8), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)])
#graph_1 = nx.Graph([(0, 4), (0, 5), (1, 7), (1, 8), (2, 4), (2, 6), (3, 5), (3, 8), (6, 7)])

In [268]:
# translate graphs into vectors 
g = nx.from_edgelist(graph_0)
degree_centrality = nx.degree_centrality(g)
feature_vec_1 = []

for i in range(n_nodes):
    feature_vec_1.append(degree_centrality.get(i))

h = nx.from_edgelist(graph_1)
degree_centrality = nx.degree_centrality(h)
feature_vec_2 = []

for i in range(n_nodes):
    feature_vec_2.append(degree_centrality.get(i))

In [269]:
# variational layer
def layer(qubits, weights):
    for j in range(qubits):
        qml.RY(weights[0][j], wires=j)
        qml.RZ(weights[1][j], wires=j)
    for k in range(qubits):
        qml.CZ(wires=[k, (k+1) % qubits])

# inverse variational layer
def inverse_layer(qubits, weights):
    for i in range(qubits):
        qml.CZ(wires= [qubits-(i+1), (qubits-i) % qubits])
    for j in range(qubits):
        qml.RZ(weights[1][j], wires=j)
        qml.RY(weights[0][j], wires=j)

In [270]:
wires = range(n_nodes)
dev = qml.device('default.qubit', n_nodes)

# list circuit
@qml.qnode(dev)
def circuit(weights, feature_vec):
    val = feature_vec
    qml.AngleEmbedding(val, wires)
    for _ in range(int(np.sqrt(n_nodes))):
        layer(n_nodes, weights)

    return qml.probs(wires)
    #return [qml.expval(qml.PauliZ(w)) for w in wires]
    #return qml.expval(qml.PauliZ(wires=[0])), qml.expval(qml.PauliZ(wires=[1]))

@qml.qnode(dev)
def inverse_circuit(weights, feature_vec1, feature_vec2):
    val1 = feature_vec1
    val2 = feature_vec2
    qml.AngleEmbedding(val1, wires)
    for _ in range(int(np.sqrt(n_nodes))):
        layer(n_nodes, weights)
    for _ in range(int(np.sqrt(n_nodes))):
        inverse_layer(n_nodes, weights)
    qml.AngleEmbedding(val2, wires)
    return [qml.expval(qml.PauliZ(w)) for w in wires]
    #return qml.expval(qml.PauliZ(wires=[0])), qml.expval(qml.PauliZ(wires=[1]))

In [271]:
# list architecture
weights = np.repeat(np.pi, 18)
weights = np.reshape(weights, (2, 9))

a = circuit(weights, feature_vec_1)[0]
b = circuit(weights, feature_vec_2)[0]

## dot product equals 1 if equal vectors, dot product equals 0 if orthogonal vectors
fidelity = np.dot(a,b)
#print(qml.draw(circuit, expansion_strategy="device")(weights, feature_vec_1))

In [272]:
def costs(weights):
    fidelity = np.dot(circuit(weights, feature_vec_1)[0], circuit(weights, feature_vec_2)[0])
    return np.array(1- fidelity)

In [273]:
np.random.seed(0)
weights_init = 0.01 * np.random.randn(2, n_nodes, requires_grad=True)
angle = [weights_init]

cost = 1 - np.dot(circuit(weights_init, feature_vec_1)[0], circuit(weights_init, feature_vec_2)[0])
opt = qml.GradientDescentOptimizer()
max_iterations = 100
conv_tol = 1e-06


### circuit results in tensor array size 2^9, but only need value at position 0 
for n in range(max_iterations):
    weights_init, prev_cost = opt.step_and_cost(circuit, weights_init, feature_vec_1)([0])
    cost.append(circuit(weights_init, feature_vec_1))
    angle.append(weights_init)

    conv = np.abs(cost[-1] - prev_cost)
    if n % 10 == 0:
        print(f"Step = {n},  Cost function = {cost[-1]:.8f} ")
    if conv <= conv_tol:
        break


TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

In [279]:
def sandwich_layer(qubits, weights):
    for j in range(int(qubits)):
        qml.RY(weights[0][j], wires=j)
        qml.RZ(weights[1][j], wires=j)
    for k in range(qubits):
        qml.CZ(wires=[k, (k+1) % qubits])

def inverse_sandwich_layer(qubits, weights):
    for i in range(qubits):
        qml.CZ(wires= [qubits-(i+1), (qubits-i) % qubits])
    for j in range(qubits):
        qml.RZ(weights[1][j], wires=j)
        qml.RY(weights[0][j], wires=j)

In [280]:
## sandwich architecture circuit
wires = range(int(np.sqrt(n_nodes)))
dev = qml.device('default.qubit', wires)

@qml.qnode(dev)
def sandwich_circuit(weights, val):
    qubits = int(np.sqrt(n_nodes))
    for i in range(qubits):
        qml.AngleEmbedding(val[i*qubits:(i*qubits)+qubits], wires)
        sandwich_layer(qubits, weights)

    return qml.probs(wires)


@qml.qnode(dev)
def inverse_sandwich_circuit(weights, feature_vec1, feature_vec2):
    val1 = feature_vec1
    val2 = feature_vec2
    
    qubits = int(np.sqrt(n_nodes))
    
    for _ in range(qubits):
        qml.AngleEmbedding(val1[i*qubits:(i*qubits)+qubits], wires)
        sandwich_layer(qubits, weights)

    for _ in range(qubits):
        inverse_sandwich_layer(qubits, weights)
        qml.AngleEmbedding(val2[i*qubits:(i*qubits)+qubits], wires)

    return [qml.expval(qml.PauliZ(w)) for w in wires]
    #return qml.expval(qml.PauliZ(wires=[0])), qml.expval(qml.PauliZ(wires=[1]))


In [281]:
# sandwich architecture
weights = np.repeat(np.pi, 6)
weights = np.reshape(weights, (2, 3))

a = sandwich_circuit(weights, feature_vec_1)[0]
b = sandwich_circuit(weights, feature_vec_2)[0]

## dot product equals 1 if equal vectors, dot product equals 0 if orthogonal vectors
fidelity = np.dot(a,b)
print(qml.draw(sandwich_circuit, expansion_strategy="device")(weights, feature_vec_1))

0: ──RX(0.12)──RY(3.14)──RZ(3.14)─╭●────╭Z──RX(0.38)──RY(3.14)──RZ(3.14)─╭●────╭Z──RX(0.75)
1: ──RX(0.38)──RY(3.14)──RZ(3.14)─╰Z─╭●─│───RX(0.38)──RY(3.14)──RZ(3.14)─╰Z─╭●─│───RX(0.88)
2: ──RX(0.38)──RY(3.14)──RZ(3.14)────╰Z─╰●──RX(0.50)──RY(3.14)──RZ(3.14)────╰Z─╰●──RX(0.25)

───RY(3.14)──RZ(3.14)─╭●────╭Z─┤ ╭Probs
───RY(3.14)──RZ(3.14)─╰Z─╭●─│──┤ ├Probs
───RY(3.14)──RZ(3.14)────╰Z─╰●─┤ ╰Probs


In [ ]:
def sandwich_costs(weights):
    fidelity = np.dot(circuit(weights, feature_vec_1)[0], circuit(weights, feature_vec_2)[0])
    return np.array(1- fidelity)

In [ ]:
np.random.seed(0)
weights_init = 0.01 * np.random.randn(2, n_nodes, requires_grad=True)
angle = [weights_init]

cost = 1 - np.dot(sandwich_circuit(weights_init, feature_vec_1)[0], sandwich_circuit(weights_init, feature_vec_2)[0])
opt = qml.GradientDescentOptimizer()
max_iterations = 100
conv_tol = 1e-06


### circuit results in tensor array size 2^9, but only need value at position 0 
for n in range(max_iterations):
    weights_init, prev_cost = opt.step_and_cost(sandwich_circuit, weights_init, feature_vec_1)([0])
    cost.append(sandwich_circuit(weights_init, feature_vec_1))
    angle.append(weights_init)

    conv = np.abs(cost[-1] - prev_cost)
    if n % 10 == 0:
        print(f"Step = {n},  Cost function = {cost[-1]:.8f} ")
    if conv <= conv_tol:
        break

In [ ]:
### TODO
### Optimierung List Circuit --> Fehlerbehebung, ich brauche nur das erste Element
### Sandwich Circuit: Gewichte anpassen, damit nicht dreimal die gleichen drankommen
### Sandwich Circuit: inverse_sandwich_circuit Amplitude Encoding in die richtige Reihenfolge bringen
### Sandwich Circuit: Optimierung